# Introduction


We are using LLama3 to write code.

# Preparation

In [1]:
from time import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

In [2]:
model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

2024-06-09 08:29:02.157943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 08:29:02.158048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 08:29:02.328277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Test function

Let's define the query model function.

In [3]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


We also define an utility function for displaying the results.

In [4]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Python code

Let's start testing the model with some Python questions.

In [5]:
system_message = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in Python to calculate the area of a rectangle with edges of L and l",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Python to calculate the area of a rectangle with edges of L and l 

**<font color='green'>Answer:</font>** Here is the Python code for the function:

```
def rectangle_area(L, l):
    return L * l
``` 

**<font color='magenta'>Total time:</font>** 4.52 sec.

In [6]:
response = query_model(
    system_message,
    user_message="Please write a function in Python to order a list",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Python to order a list 

**<font color='green'>Answer:</font>** Here is a Python function that orders a list:

```
def order_list(lst):
    return sorted(lst)
``` 

**<font color='magenta'>Total time:</font>** 1.69 sec.

In [7]:
response = query_model(
    system_message,
    user_message="Please write a Python module for a phone agenda.", 
    temperature=0.1,
    max_length=1024)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a Python module for a phone agenda. 

**<font color='green'>Answer:</font>** Here is a simple Python module for a phone agenda:

```
class PhoneAgenda:
    def __init__(self):
        self.contacts = {}

    def add_contact(self, name, phone_number):
        self.contacts[name] = phone_number

    def delete_contact(self, name):
        if name in self.contacts:
            del self.contacts[name]
        else:
            print("Contact not found")

    def update_contact(self, name, phone_number):
        if name in self.contacts:
            self.contacts[name] = phone_number
        else:
            print("Contact not found")

    def search_contact(self, name):
        if name in self.contacts:
            return self.contacts[name]
        else:
            return "Contact not found"

    def display_contacts(self):
        for name, phone_number in self.contacts.items():
            print(f"Name: {name}, Phone Number: {phone_number}")


def main():
    agenda = PhoneAgenda()

    while True:
        print("\nPhone Agenda Menu:")
        print("1. Add Contact")
        print("2. Delete Contact")
        print("3. Update Contact")
        print("4. Search Contact")
        print("5. Display Contacts")
        print("6. Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            name = input("Enter contact name: ")
            phone_number = input("Enter contact phone number: ")
            agenda.add_contact(name, phone_number)
        elif choice == "2":
            name = input("Enter contact name: ")
            agenda.delete_contact(name)
        elif choice == "3":
            name = input("Enter contact name: ")
            phone_number = input("Enter new phone number: ")
            agenda.update_contact(name, phone_number)
        elif choice == "4":
            name = input("Enter contact name: ")
            print(agenda.search_contact(name))
        elif choice == "5":
            agenda.display_contacts()
        elif choice == "6":
            break
        else:
            print("Invalid choice. Please choose a valid option.")


if __name__ == "__main__":
    main()
``` 

**<font color='magenta'>Total time:</font>** 30.39 sec.

In [8]:
response = query_model(
    system_message,
    user_message="""
                            Please write a Python service using Flask to expose a machine learning model. 
                            The service has one endpoint (POST) that receives three features:
                            - area
                            - number of rooms
                            - number of bathrooms
                            The model returns the price of the property.""",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
                            Please write a Python service using Flask to expose a machine learning model. 
                            The service has one endpoint (POST) that receives three features:
                            - area
                            - number of rooms
                            - number of bathrooms
                            The model returns the price of the property. 

**<font color='green'>Answer:</font>** Here is a simple Python service using Flask to expose a machine learning model:

```
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)

# Load the dataset
df = pd.read_csv('property_data.csv')

# Split the data into features and target
X = df.drop('price', axis=1)
y = df['price']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train the model
model = RandomForestRegressor()
model.fit(X_scaled, y)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    area = data['area']
    num_rooms = data['num_rooms']
    num_bathrooms = data['num_bathrooms']

    # Scale the input data
    input_data = [[area, num_rooms, num_bathrooms]]
    input_data_scaled = scaler.transform(input_data)

    # Make the prediction
    prediction = model.predict(input_data_scaled)[0]

    return jsonify({'price': prediction})

if __name__ == '__main__':
    app.run(debug=True)
``` 

**<font color='magenta'>Total time:</font>** 17.42 sec.

# C++ code

Let's continue with few C++ questions.

In [9]:
system_message = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in C++ to calculate the area of a rectangle with edges of L and l",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to calculate the area of a rectangle with edges of L and l 

**<font color='green'>Answer:</font>** Here is the C++ code for the function:

```cpp
#include<iostream>
using namespace std;

float area(float L, float l) {
    return L * l;
}

int main() {
    float L, l;
    cout << "Enter the length of the rectangle: ";
    cin >> L;
    cout << "Enter the breadth of the rectangle: ";
    cin >> l;
    cout << "The area of the rectangle is: " << area(L, l);
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 7.43 sec.

In [10]:
response = query_model(
    system_message,
    user_message="Please write a function in C++ to traverse a balanced tree",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to traverse a balanced tree 

**<font color='green'>Answer:</font>** Here is a simple C++ function to traverse a balanced binary tree in-order, pre-order, and post-order:

```cpp
#include <iostream>
using namespace std;

// Define the structure for a node in the tree
struct Node {
    int data;
    Node* left;
    Node* right;
};

// Function to create a new node
Node* createNode(int data) {
    Node* newNode = new Node();
    if (!newNode) {
        cout << "Memory error\n";
        return NULL;
    }
    newNode->data = data;
    newNode->left = newNode->right = NULL;
    return newNode;
}

// Function to perform in-order traversal
void inOrderTraversal(Node* root) {
    if (root == NULL) return;
    inOrderTraversal(root->left);
    cout << root->data << " ";
    inOrderTraversal(root->right);
}

// Function to perform pre-order traversal
void preOrderTraversal(Node* root) {
    if (root == NULL) return;
    cout << root->data << " ";
    preOrderTraversal(root->left);
    preOrderTraversal(root->right);
}

// Function to perform post-order traversal
void postOrderTraversal(Node* root) {
    if (root == NULL) return;
    postOrderTraversal(root->left);
    postOrderTraversal(root->right);
    cout << root->data << " ";
}

int main() {
    // Create a sample balanced binary tree
    Node* root = createNode(1);
    root->left = createNode(2);
    root->right = createNode(3);
    root->left->left = createNode(4);
    root->left->right = createNode(5);

    // Perform in-order traversal
    cout << "In-order traversal: ";
    inOrderTraversal(root);
    cout << endl;

    // Perform pre-order traversal
    cout << "Pre-order traversal: ";
    preOrderTraversal(root);
    cout << endl;

    // Perform post-order traversal
    cout << "Post-order traversal: ";
    postOrderTraversal(root);
    cout << endl;

    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 30.93 sec.

In [11]:
response = query_model(
    system_message,
    user_message="Please write a class in C++ for allocation of small objects",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a class in C++ for allocation of small objects 

**<font color='green'>Answer:</font>** Here is a simple implementation of a class in C++ for allocation of small objects using a pool allocator:

```cpp
#include <iostream>
#include <vector>
#include <memory>

class SmallObjectAllocator {
private:
    std::vector<std::unique_ptr<void>> pool;

public:
    SmallObjectAllocator(size_t size) {
        pool.resize(size);
    }

    template <typename T>
    T* allocate() {
        for (auto& ptr : pool) {
            if (!ptr) {
                ptr.reset(new T());
                return static_cast<T*>(ptr.get());
            }
        }
        return nullptr;
    }

    void deallocate(T* ptr) {
        for (auto& p : pool) {
            if (p.get() == ptr) {
                p.reset();
                return;
            }
        }
    }
};

int main() {
    SmallObjectAllocator allocator(10);

    for (int i = 0; i < 5; ++i) {
        int* p = static_cast<int*>(allocator.allocate<int>());
        *p = i;
        std::cout << *p << std::endl;
    }

    allocator.deallocate(static_cast<int*>(allocator.allocate<int>()));

    return 0;
}
```

This code creates a pool of small objects and provides methods to allocate and deallocate objects from the pool. The `allocate` method returns a pointer to a newly allocated object if available in the pool, otherwise it returns `nullptr`. The `deallocate` method resets the pointer to the object in the pool, effectively freeing the memory. 

**<font color='magenta'>Total time:</font>** 22.32 sec.

# SQL code

Let's continue by generating some transactional SQL.

In [12]:
system_message = """
You are an AI assistant designed to write simple SQL code.
Please answer with the listing of the SQL code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a small stored procedure in SQL to debit one account and credit another account.",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a small stored procedure in SQL to debit one account and credit another account. 

**<font color='green'>Answer:</font>** Here is a sample stored procedure in SQL to debit one account and credit another account:

```
CREATE PROCEDURE DebitAndCredit
    @account_to_debit int,
    @account_to_credit int,
    @amount decimal(10,2)
AS
BEGIN
    BEGIN TRANSACTION
    BEGIN TRY
        UPDATE Accounts
        SET Balance = Balance - @amount
        WHERE AccountID = @account_to_debit

        UPDATE Accounts
        SET Balance = Balance + @amount
        WHERE AccountID = @account_to_credit

        COMMIT TRANSACTION
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION
        THROW
    END CATCH
END
``` 

**<font color='magenta'>Total time:</font>** 9.86 sec.

# Erlang code


Let's continue by generating some Erlang code.

In [13]:
system_message = """
You are an AI assistant designed to write simple Erlang code.
Please answer with the listing of the Erlang code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a small module in Erlang for sending transactions between two parties.",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a small module in Erlang for sending transactions between two parties. 

**<font color='green'>Answer:</font>** Here is a simple Erlang module that simulates sending transactions between two parties:

```erlang
-module(transaction).
-export([start/0, send/2, receive/1]).

% Initialize the balance for both parties
party1_balance = 1000.
party2_balance = 1000.

start() ->
    io:format("Transaction system started.~n").

send(Party, Amount) ->
    case Party of
        party1 ->
            if Amount =< party1_balance ->
                party1_balance = party1_balance - Amount,
                party2_balance = party2_balance + Amount,
                io:format("Party 1 sent ~p to Party 2.~n", [Amount]);
               _ ->
                io:format("Party 1 does not have enough balance.~n")
            end;
        party2 ->
            if Amount =< party2_balance ->
                party1_balance = party1_balance + Amount,
                party2_balance = party2_balance - Amount,
                io:format("Party 2 sent ~p to Party 1.~n", [Amount]);
               _ ->
                io:format("Party 2 does not have enough balance.~n")
            end;
        _ ->
            io:format("Invalid party.~n")
    end.

receive(Party) ->
    case Party of
        party1 ->
            io:format("Party 1 balance: ~p~n", [party1_balance]);
        party2 ->
            io:format("Party 2 balance: ~p~n", [party2_balance]);
        _ ->
            io:format("Invalid party.~n")
    end.
``` 

**<font color='magenta'>Total time:</font>** 23.81 sec.

# Ada code

Let's write some Ada code.

In [14]:
system_message = """
You are an AI assistant designed to write simple Ada code.
Please answer with the listing of the Ada code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in Ada to calculate the area of a rectangle with edges of L and l",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Ada to calculate the area of a rectangle with edges of L and l 

**<font color='green'>Answer:</font>** Here is the Ada code to calculate the area of a rectangle:

```ada
function Rectangle_Area (L, l : Float) return Float is
   Area : Float;
begin
   Area := L * l;
   return Area;
end Rectangle_Area;
``` 

**<font color='magenta'>Total time:</font>** 3.97 sec.

# FORTRAN code

Let's write some FORTRAN code.

In [15]:
system_message = """
You are an AI assistant designed to write simple FORTRAN code.
Please answer with the listing of the FORTRAN code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in FORTRAN to calculate the area of a rectangle with edges of L and l",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 104



**<font color='red'>Question:</font>** Please write a function in FORTRAN to calculate the area of a rectangle with edges of L and l 

**<font color='green'>Answer:</font>** PROGRAM AREA_OF_RECTANGLE
  IMPLICIT NONE
  REAL :: L, l, area

  PRINT *, "Enter the length of the rectangle: "
  READ *, L
  PRINT *, "Enter the width of the rectangle: "
  READ *, l

  area = L * l

  PRINT *, "The area of the rectangle is: ", area

END PROGRAM AREA_OF_RECTANGLE 

**<font color='magenta'>Total time:</font>** 5.72 sec.

# COBOL code

Let's try some COBOL code.

In [16]:
system_message = """
You are an AI assistant designed to write simple COBOL code.
Please answer with the listing of the COBOL code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="""Please write a function in COBOL to calculate the average transaction amount during current month,
    given the amounts of all the transactions and the dates of the transactions.""",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in COBOL to calculate the average transaction amount during current month,
    given the amounts of all the transactions and the dates of the transactions. 

**<font color='green'>Answer:</font>** IDENTIFICATION DIVISION.
PROGRAM-ID. AVG-TRANSACTION-AMOUNT.

DATA DIVISION.
WORKING-STORAGE SECTION.
01  TRANSACTION-AMOUNTS OCCURS 100 TIMES PIC 9(5).
01  TRANSACTION-DATES OCCURS 100 TIMES PIC X(10).
01  CURRENT-MONTH PIC 99.
01  TOTAL-AMOUNT PIC 9(7) VALUE 0.
01  TRANSACTION-COUNT PIC 9(2) VALUE 0.
01  AVERAGE-AMOUNT PIC 9(5) VALUE 0.

PROCEDURE DIVISION.
    PERFORM GET-TRANSACTION-DATA
    PERFORM CALCULATE-AVERAGE
    DISPLAY AVERAGE-AMOUNT
    STOP RUN.

GET-TRANSACTION-DATA.
    PERFORM VARYING I FROM 1 BY 1 UNTIL I > 100
        ACCEPT "Enter transaction amount " I ": " WITH NO ADVANCING
        ACCEPT "Enter transaction date " I ": " WITH NO ADVANCING
        MOVE FUNCTION CURRENT-DATE TO TRANSACTION-DATES(I)
        ADD TRANSACTION-AMOUNTS(I) TO TOTAL-AMOUNT
        ADD 1 TO TRANSACTION-COUNT
    END-PERFORM.

 

**<font color='magenta'>Total time:</font>** 17.83 sec.

# Conclusion

Llama3 proved to be proficient while writing code in a large number of languages, including objectual code, procedural, functional, and directive.